In [189]:
import pandas as pd
import numpy as np
skiprows = np.random.rand(55 * 10 ** 7) > 0.02
skiprows[0] = False
df = pd.read_csv('train.csv', skiprows=lambda x: skiprows[x])
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05 UTC,-73.979279,40.723438,-74.004608,40.746948,6
1,2011-01-05 17:39:00.00000040,3.7,2011-01-05 17:39:00 UTC,-73.966957,40.761268,-73.967912,40.765535,1
2,2013-09-17 04:22:00.0000006,19.0,2013-09-17 04:22:00 UTC,-73.987210,40.729325,-73.931985,40.697207,1
3,2011-03-09 22:21:25.0000001,9.7,2011-03-09 22:21:25 UTC,-73.977829,40.788979,-73.967935,40.760508,1
4,2009-10-16 10:32:00.000000191,7.3,2009-10-16 10:32:00 UTC,-73.990575,40.746117,-74.003227,40.751447,1


In [190]:
df.shape

(1108485, 8)

In [191]:
df1 = pd.read_csv('test.csv')
df1.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [193]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'],utc=True)
df1['pickup_datetime'] = pd.to_datetime(df1['pickup_datetime'],utc=True)

In [41]:
X_train = df.iloc[:,2:]
Y_train = df['fare_amount']

In [42]:
X_test = df1.iloc[:,1:]

In [68]:
X_train['pickup_datetime'][3].dayofweek

5

In [54]:
!pip install holidays

  Created wheel for holidays: filename=holidays-0.10.5.2-py3-none-any.whl size=126818 sha256=e3b6c605924bd9e3e5b8e0100aef14af70b13b0c4d7f0a8124b9e6255af180fe
  Stored in directory: c:\users\rahul\appdata\local\pip\cache\wheels\6a\cf\7a\3ffc6ba5930fd024335b4a12b0be8bc3ba5a8fa63ca9a849ef
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730977 sha256=c2d11d38dac7511f48f748ccc2588babc0ab35494a536e7ad9435ed5665da52d
  Stored in directory: c:\users\rahul\appdata\local\pip\cache\wheels\a0\8b\b2\810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built holidays pymeeus


In [67]:
!pip install haversine

In [188]:
import haversine as hs
from tqdm import tqdm
df['distance_travelled'] = None
df['pickup_location'] = list(zip(df.pickup_latitude, df.pickup_longitude))
df['dropoff_location'] = list(zip(df.dropoff_latitude, df.dropoff_longitude))
for i in tqdm(range(df.shape[0])):
    df['distance_travelled'][i] = hs.haversine(df['pickup_location'][i],df['dropoff_location'][i])
    df['is_a_holiday'][i] = df['pickup_datetime'][i] in us_holidays
df['is_a_holiday'].apply(int)

  0%|                                                                                      | 0/1110767 [00:00<?, ?it/s]<ipython-input-188-5c51f4faf8d5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_travelled'][i] = hs.haversine(df['pickup_location'][i],df['dropoff_location'][i])
<ipython-input-188-5c51f4faf8d5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_a_holiday'][i] = df['pickup_datetime'][i] in us_holidays
  0%|▏                                                                       | 2566/1110767 [00:16<2:01:06, 152.51it/s]


KeyboardInterrupt: 

In [197]:
import haversine as hs
from tqdm import tqdm
df['distance'] = list(zip(list(zip(df.pickup_latitude, df.pickup_longitude)),list(zip(df.dropoff_latitude, df.dropoff_longitude))))
df['distance_travelled'] = df['distance'].apply(lambda x: hs.haversine(x[0],x[1]))
df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
df['month'] = df['pickup_datetime'].apply(lambda x:x.month)
df['day'] = df['pickup_datetime'].apply(lambda x: x.dayofweek)
df["befor_shock"] = ((df["year"] <= 2011) | ((df["year"] <= 2012) & (df["month"] <= 8))).apply(int)
df['is_a_holiday'] = (df['pickup_datetime'].apply(lambda x: x in us_holidays)).apply(int)
df['passenger_count'] = (df['passenger_count']<5).apply(int)
df["time"] = df["pickup_datetime"].apply(lambda x: x.hour * 60 + x.minute)

In [199]:
df1['distance'] = list(zip(list(zip(df1.pickup_latitude, df1.pickup_longitude)),list(zip(df1.dropoff_latitude, df1.dropoff_longitude))))
df1['distance_travelled'] = df1['distance'].apply(lambda x: hs.haversine(x[0],x[1]))
df1['year'] = df1['pickup_datetime'].apply(lambda x: x.year)
df1['month'] = df1['pickup_datetime'].apply(lambda x:x.month)
df1['day'] = df1['pickup_datetime'].apply(lambda x: x.dayofweek)
df1["befor_shock"] = ((df1["year"] <= 2011) | ((df1["year"] <= 2012) & (df1["month"] <= 8))).apply(int)
df1['is_a_holiday'] = (df1['pickup_datetime'].apply(lambda x: x in us_holidays)).apply(int)
df1['passenger_count'] = (df1['passenger_count']<5).apply(int)
df1["time"] = df1["pickup_datetime"].apply(lambda x: x.hour * 60 + x.minute)

In [198]:
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,distance_travelled,year,month,day,befor_shock,is_a_holiday,time
0,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,0,"((40.72343826293945, -73.97927856445312), (40....",3.374718,2015,4,6,0,0,619
1,2011-01-05 17:39:00.00000040,3.7,2011-01-05 17:39:00+00:00,-73.966957,40.761268,-73.967912,40.765535,1,"((40.761268, -73.966957), (40.765535, -73.9679...",0.481238,2011,1,2,1,0,1059
2,2013-09-17 04:22:00.0000006,19.0,2013-09-17 04:22:00+00:00,-73.987210,40.729325,-73.931985,40.697207,1,"((40.729325, -73.98721), (40.697207, -73.931985))",5.866839,2013,9,1,0,0,262
3,2011-03-09 22:21:25.0000001,9.7,2011-03-09 22:21:25+00:00,-73.977829,40.788979,-73.967935,40.760508,1,"((40.788979, -73.977829), (40.760508, -73.9679...",3.273626,2011,3,2,1,0,1341
4,2009-10-16 10:32:00.000000191,7.3,2009-10-16 10:32:00+00:00,-73.990575,40.746117,-74.003227,40.751447,1,"((40.746117, -73.990575), (40.751447, -74.0032...",1.219496,2009,10,4,1,0,632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108480,2009-04-30 21:37:00.00000040,10.5,2009-04-30 21:37:00+00:00,-73.997763,40.741155,-73.984755,40.717655,1,"((40.741155, -73.997763), (40.717655, -73.9847...",2.833663,2009,4,3,1,0,1297
1108481,2012-08-03 23:31:00.000000184,3.3,2012-08-03 23:31:00+00:00,-73.967117,40.756930,-73.973147,40.755955,1,"((40.75693, -73.967117), (40.755955, -73.973147))",0.519345,2012,8,4,1,0,1411
1108482,2011-02-20 14:03:00.00000051,15.3,2011-02-20 14:03:00+00:00,-73.991902,40.738360,-73.967693,40.792793,1,"((40.73836, -73.991902), (40.792792999999996, ...",6.386844,2011,2,6,1,0,843
1108483,2011-08-06 18:36:47.0000001,7.7,2011-08-06 18:36:47+00:00,-73.992143,40.770590,-73.988171,40.758195,1,"((40.770590000000006, -73.992143), (40.758195,...",1.418278,2011,8,5,1,0,1116


In [150]:
day_dummy=pd.get_dummies(df['day'])

#df=pd.concat([df,train_dummy],axis=1)
#holiday_dummy = pd.get_dummies(df['is_a_holiday'])
#passenger_dummy = pd.get_dummies(df['passeger_count'])
#df=pd.concat([df,day_dummy,holiday_dummy, passenger_dummy],axis=1)
df=pd.concat([df,day_dummy],axis=1)

day1_dummy=pd.get_dummies(df1['day'])
#df=pd.concat([df,train_dummy],axis=1)
#holiday_dummy = pd.get_dummies(df['is_a_holiday'])
#passenger_dummy = pd.get_dummies(df['passeger_count'])
#df=pd.concat([df,day_dummy,holiday_dummy, passenger_dummy],axis=1)
df1=pd.concat([df1,day1_dummy],axis=1)

In [213]:
def oneHotEncode(df,colNames):
    for col in colNames:
        dummies = pd.get_dummies(df[col],prefix=col)
        df = pd.concat([df,dummies],axis=1)

        #drop the encoded column
        df.drop([col],axis = 1 , inplace=True)
    return df
    
cat_cols = ['is_a_holiday','befor_shock','day','month','year','passenger_count']
print('There were {} columns before encoding categorical features'.format(df.shape[1]))
df = oneHotEncode(df, cat_cols)
df1 = oneHotEncode(df1, cat_cols)
print('There are {} columns after encoding categorical features'.format(df.shape[1]))

There were 16 columns before encoding categorical features
There are 42 columns after encoding categorical features


In [200]:
df.dropna(inplace=True)

In [226]:
X_train = df.drop(columns = ['key','fare_amount','distance','pickup_datetime'])
y_train = df['fare_amount']
X_test = df1.drop(columns = ['key','distance','pickup_datetime'])

In [227]:
y_train

0          11.5
1           3.7
2          19.0
3           9.7
4           7.3
           ... 
1108480    10.5
1108481     3.3
1108482    15.3
1108483     7.7
1108484    14.1
Name: fare_amount, Length: 1108472, dtype: float64

In [151]:
dftest = df1.iloc[:,-13:]
dftest.drop(columns = ['day','dropoff_location','pickup_location'],inplace=True)

In [142]:
dftrain.dropna(inplace=True)

In [152]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(dftrain,df['fare_amount'])
y_pred = regr.predict(dftest)

In [6]:
df2 = pd.read_csv('sample_submission.csv')
df2.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.35
1,2015-01-27 13:08:24.0000003,11.35
2,2011-10-08 11:53:44.0000002,11.35
3,2012-12-01 21:12:12.0000002,11.35
4,2012-12-01 21:12:12.0000003,11.35


In [157]:
y_pred.min()

3.3711217078712763

In [263]:
Submission=pd.DataFrame(data=y_pred,columns=['fare_amount'])
Submission['key']=df1['key']
Submission=Submission[['key','fare_amount']]

In [264]:
Submission = Submission.set_index('key')

In [265]:
Submission.to_csv('y1_knn.csv')

In [250]:
Submission.head()

,fare_amount
key,
2015-01-27 13:08:24.0000002,11.292572
2015-01-27 13:08:24.0000003,10.649523
2011-10-08 11:53:44.0000002,4.736473
2012-12-01 21:12:12.0000002,10.116229
2012-12-01 21:12:12.0000003,15.810888


In [246]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=30, n_estimators=100, n_jobs=-1)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [251]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [220]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 

In [222]:

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               4992      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 169,857
Trainable params: 169,857
Non-trainable params: 0
_________________________________________________________________


In [238]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [239]:
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
27699/27712 [============================>.] - ETA: 0s - loss: 2.0851 - mean_absolute_error: 2.0851
Epoch 00001: val_loss improved from inf to 2.10555, saving model to Weights-001--2.10555.hdf5
27712/27712 [==============================] - 36s 1ms/step - loss: 2.0851 - mean_absolute_error: 2.0851 - val_loss: 2.1055 - val_mean_absolute_error: 2.1055
Epoch 2/500
27711/27712 [============================>.] - ETA: 0s - loss: 2.0852 - mean_absolute_error: 2.0852
Epoch 00002: val_loss did not improve from 2.10555
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0852 - mean_absolute_error: 2.0852 - val_loss: 2.1553 - val_mean_absolute_error: 2.1553
Epoch 3/500
27700/27712 [============================>.] - ETA: 0s - loss: 2.0866 - mean_absolute_error: 2.0866
Epoch 00003: val_loss improved from 2.10555 to 2.09226, saving model to Weights-003--2.09226.hdf5
27712/27712 [==============================] - 41s 1ms/step - loss: 2.0866 - mean_absolute_error: 2.0866 

Epoch 26/500
27701/27712 [============================>.] - ETA: 0s - loss: 2.0847 - mean_absolute_error: 2.0847
Epoch 00026: val_loss did not improve from 2.07691
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0849 - mean_absolute_error: 2.0849 - val_loss: 2.1512 - val_mean_absolute_error: 2.1512
Epoch 27/500
27703/27712 [============================>.] - ETA: 0s - loss: 2.0860 - mean_absolute_error: 2.0860
Epoch 00027: val_loss did not improve from 2.07691
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0860 - mean_absolute_error: 2.0860 - val_loss: 2.1021 - val_mean_absolute_error: 2.1021
Epoch 28/500
27691/27712 [============================>.] - ETA: 0s - loss: 2.0852 - mean_absolute_error: 2.0852
Epoch 00028: val_loss did not improve from 2.07691
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0852 - mean_absolute_error: 2.0852 - val_loss: 2.1117 - val_mean_absolute_error: 2.1117
Epoch 29/500
27685/27712 [========

27691/27712 [============================>.] - ETA: 0s - loss: 2.0821 - mean_absolute_error: 2.0821
Epoch 00051: val_loss did not improve from 2.07546
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0823 - mean_absolute_error: 2.0823 - val_loss: 2.0992 - val_mean_absolute_error: 2.0992
Epoch 52/500
27688/27712 [============================>.] - ETA: 0s - loss: 2.0837 - mean_absolute_error: 2.0837
Epoch 00052: val_loss did not improve from 2.07546
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0835 - mean_absolute_error: 2.0835 - val_loss: 2.0817 - val_mean_absolute_error: 2.0817
Epoch 53/500
27694/27712 [============================>.] - ETA: 0s - loss: 2.0827 - mean_absolute_error: 2.0827
Epoch 00053: val_loss did not improve from 2.07546
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0827 - mean_absolute_error: 2.0827 - val_loss: 2.0859 - val_mean_absolute_error: 2.0859
Epoch 54/500
27707/27712 [=====================

27696/27712 [============================>.] - ETA: 0s - loss: 2.0794 - mean_absolute_error: 2.0794
Epoch 00076: val_loss improved from 2.07296 to 2.06861, saving model to Weights-076--2.06861.hdf5
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0795 - mean_absolute_error: 2.0795 - val_loss: 2.0686 - val_mean_absolute_error: 2.0686
Epoch 77/500
27706/27712 [============================>.] - ETA: 0s - loss: 2.0797 - mean_absolute_error: 2.0797
Epoch 00077: val_loss did not improve from 2.06861
27712/27712 [==============================] - 38s 1ms/step - loss: 2.0796 - mean_absolute_error: 2.0796 - val_loss: 2.0704 - val_mean_absolute_error: 2.0704
Epoch 78/500
27708/27712 [============================>.] - ETA: 0s - loss: 2.0795 - mean_absolute_error: 2.0795
Epoch 00078: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0793 - mean_absolute_error: 2.0793 - val_loss: 2.0772 - val_mean_absolute_error: 2.0772


27701/27712 [============================>.] - ETA: 0s - loss: 2.0786 - mean_absolute_error: 2.0786
Epoch 00101: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0789 - mean_absolute_error: 2.0789 - val_loss: 2.0721 - val_mean_absolute_error: 2.0721
Epoch 102/500
27672/27712 [============================>.] - ETA: 0s - loss: 2.0787 - mean_absolute_error: 2.0787
Epoch 00102: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0789 - mean_absolute_error: 2.0789 - val_loss: 2.0861 - val_mean_absolute_error: 2.0861
Epoch 103/500
27682/27712 [============================>.] - ETA: 0s - loss: 2.0790 - mean_absolute_error: 2.0790
Epoch 00103: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0788 - mean_absolute_error: 2.0788 - val_loss: 2.0757 - val_mean_absolute_error: 2.0757
Epoch 104/500
27676/27712 [==================

...

Epoch 385/500
27687/27712 [============================>.] - ETA: 0s - loss: 2.0753 - mean_absolute_error: 2.0753
Epoch 00385: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0754 - mean_absolute_error: 2.0754 - val_loss: 2.0897 - val_mean_absolute_error: 2.0897
Epoch 386/500
27689/27712 [============================>.] - ETA: 0s - loss: 2.0744 - mean_absolute_error: 2.0744
Epoch 00386: val_loss did not improve from 2.06861
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0743 - mean_absolute_error: 2.0743 - val_loss: 2.0850 - val_mean_absolute_error: 2.0850
Epoch 387/500
27707/27712 [============================>.] - ETA: 0s - loss: 2.0755 - mean_absolute_error: 2.0755
Epoch 00387: val_loss did not improve from 2.06861
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0755 - mean_absolute_error: 2.0755 - val_loss: 2.0954 - val_mean_absolute_error: 2.0954
Epoch 388/500
27687/27712 [====

Epoch 411/500
27686/27712 [============================>.] - ETA: 0s - loss: 2.0761 - mean_absolute_error: 2.0761
Epoch 00411: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0761 - mean_absolute_error: 2.0761 - val_loss: 2.1044 - val_mean_absolute_error: 2.1044
Epoch 412/500
27705/27712 [============================>.] - ETA: 0s - loss: 2.0750 - mean_absolute_error: 2.0750
Epoch 00412: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0750 - mean_absolute_error: 2.0750 - val_loss: 2.0726 - val_mean_absolute_error: 2.0726
Epoch 413/500
27688/27712 [============================>.] - ETA: 0s - loss: 2.0751 - mean_absolute_error: 2.0751
Epoch 00413: val_loss did not improve from 2.06861
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0752 - mean_absolute_error: 2.0752 - val_loss: 2.0808 - val_mean_absolute_error: 2.0808
Epoch 414/500
27692/27712 [====

27709/27712 [============================>.] - ETA: 0s - loss: 2.0746 - mean_absolute_error: 2.0746
Epoch 00436: val_loss did not improve from 2.06861
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0746 - mean_absolute_error: 2.0746 - val_loss: 2.0894 - val_mean_absolute_error: 2.0894
Epoch 437/500
27702/27712 [============================>.] - ETA: 0s - loss: 2.0748 - mean_absolute_error: 2.0748
Epoch 00437: val_loss did not improve from 2.06861
27712/27712 [==============================] - 39s 1ms/step - loss: 2.0749 - mean_absolute_error: 2.0749 - val_loss: 2.0778 - val_mean_absolute_error: 2.0778
Epoch 438/500
27685/27712 [============================>.] - ETA: 0s - loss: 2.0749 - mean_absolute_error: 2.0749
Epoch 00438: val_loss did not improve from 2.06861
27712/27712 [==============================] - 40s 1ms/step - loss: 2.0748 - mean_absolute_error: 2.0748 - val_loss: 2.0728 - val_mean_absolute_error: 2.0728
Epoch 439/500
27699/27712 [==================

Epoch 462/500
27703/27712 [============================>.] - ETA: 0s - loss: 2.0816 - mean_absolute_error: 2.0816
Epoch 00462: val_loss did not improve from 2.06861
27712/27712 [==============================] - 42s 1ms/step - loss: 2.0816 - mean_absolute_error: 2.0816 - val_loss: 2.0936 - val_mean_absolute_error: 2.0936
Epoch 463/500
27712/27712 [==============================] - ETA: 0s - loss: 2.0826 - mean_absolute_error: 2.0826
Epoch 00463: val_loss did not improve from 2.06861
27712/27712 [==============================] - 42s 2ms/step - loss: 2.0826 - mean_absolute_error: 2.0826 - val_loss: 2.0822 - val_mean_absolute_error: 2.0822
Epoch 464/500
27709/27712 [============================>.] - ETA: 0s - loss: 2.0804 - mean_absolute_error: 2.0804
Epoch 00464: val_loss did not improve from 2.06861
27712/27712 [==============================] - 42s 2ms/step - loss: 2.0804 - mean_absolute_error: 2.0804 - val_loss: 2.0756 - val_mean_absolute_error: 2.0756
Epoch 465/500
27684/27712 [====

Epoch 488/500
27694/27712 [============================>.] - ETA: 0s - loss: 2.0777 - mean_absolute_error: 2.0777
Epoch 00488: val_loss did not improve from 2.06861
27712/27712 [==============================] - 46s 2ms/step - loss: 2.0779 - mean_absolute_error: 2.0779 - val_loss: 2.1058 - val_mean_absolute_error: 2.1058
Epoch 489/500
27706/27712 [============================>.] - ETA: 0s - loss: 2.0782 - mean_absolute_error: 2.0782
Epoch 00489: val_loss did not improve from 2.06861
27712/27712 [==============================] - 46s 2ms/step - loss: 2.0781 - mean_absolute_error: 2.0781 - val_loss: 2.0799 - val_mean_absolute_error: 2.0799
Epoch 490/500
27691/27712 [============================>.] - ETA: 0s - loss: 2.0813 - mean_absolute_error: 2.0813
Epoch 00490: val_loss did not improve from 2.06861
27712/27712 [==============================] - 46s 2ms/step - loss: 2.0813 - mean_absolute_error: 2.0813 - val_loss: 2.1054 - val_mean_absolute_error: 2.1054
Epoch 491/500
27701/27712 [====

In [240]:
wights_file = 'Weights-076--2.06861.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [241]:
y_pred = NN_model.predict(X_test)

In [242]:
y_pred

array([[10.418053 ],
       [10.769339 ],
       [ 4.0847797],
       ...,
       [55.06902  ],
       [22.621426 ],
       [ 6.283396 ]], dtype=float32)

In [257]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
def knn_model(x_train, x_val, y_train, y_val, neighbors):
    min_rmse = 1000
    for n in neighbors:
        knn = KNeighborsRegressor(n_neighbors=n)
        knn.fit(x_train, y_train)
        pred = knn.predict(x_val)
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        if rmse < min_rmse:
            min_rmse = rmse
            model = knn
            best_pred = pred
        print('Neighbours', n, 'RMSE', rmse)
    return model, min_rmse, best_pred

In [258]:
k_choices = [18,24,30,40]
knn_final_model, knn_final_rmse, knn_final_pred = knn_model(X_train, X_val, y_train, y_val, k_choices)
knn_test_pred = knn_final_model.predict(X_test)

Neighbours 18 RMSE 5.1377800001928735
Neighbours 24 RMSE 5.12771491074715
Neighbours 30 RMSE 5.127487618943662
Neighbours 40 RMSE 5.130315314887339


In [262]:
y_pred = knn_test_pred